In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: vinay7410
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews


100%|██████████| 25.7M/25.7M [00:00<00:00, 995MB/s]

In [3]:
import os
import json
import tensorflow
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
data=pd.read_csv("/content/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [5]:
data.shape

(50000, 2)

In [6]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [8]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data["sentiment"]=le.fit_transform(data["sentiment"])

In [9]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [10]:
train_data, test_data=train_test_split(data, test_size=0.2, random_state=42)
print(train_data.shape, test_data.shape)

(40000, 2) (10000, 2)


In [11]:
tokeniser=Tokenizer(num_words=5000)
tokeniser.fit_on_texts(train_data["review"])
X_train=pad_sequences(tokeniser.texts_to_sequences(train_data["review"]), maxlen=200)
X_test=pad_sequences(tokeniser.texts_to_sequences(test_data["review"]), maxlen=200)

In [20]:
X_train.shape

(40000, 200)

In [14]:
Y_train=train_data["sentiment"]
Y_test=test_data["sentiment"]

In [15]:
Y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


In [16]:
model =Sequential()

model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation="sigmoid"))

# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
model.fit(X_train, Y_train, batch_size=64, epochs=5, validation_split=0.2)


Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 204s 392ms/step - accuracy: 0.6975 - loss: 0.5633 - val_accuracy: 0.8034 - val_loss: 0.4323
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 406ms/step - accuracy: 0.8144 - loss: 0.4218 - val_accuracy: 0.8504 - val_loss: 0.3699
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 250s 382ms/step - accuracy: 0.8608 - loss: 0.3408 - val_accuracy: 0.8633 - val_loss: 0.3469
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 217s 433ms/step - accuracy: 0.8852 - loss: 0.2875 - val_accuracy: 0.8798 - val_loss: 0.3015
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 241s 392ms/step - accuracy: 0.9084 - loss: 0.2394 - val_accuracy: 0.8802 - val_loss: 0.3038


In [30]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (64, 200, 128)         │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (64, 128)              │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,315,141 (8.83 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,543,428 (5.89 MB)

In [21]:
loss,accuracy=model.evaluate(X_test, Y_test)
print(f"Test loss :{loss} ")
print(f"Test accuracy :{accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 36s 114ms/step - accuracy: 0.8803 - loss: 0.2938
Test loss :0.2935296595096588 
Test accuracy :0.8812000155448914


In [22]:
def predict_ssentiment(review):
  sequence=tokeniser.texts_to_sequences([review])
  padded_sequences=pad_sequences(sequence, maxlen=200)
  prediction=model.predict(padded_sequences)
  sentiment="positive" if prediction[0][0]>0.5 else "negative"

  return sentiment

In [23]:
new_review="The movie was fantastic ,I loved it."
sentiment=predict_ssentiment(new_review)
print(f"The sentiment of the review is {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 423ms/step
The sentiment of the review is positive


In [25]:
new_review="The movie was bad."
sentiment=predict_ssentiment(new_review)
print(f"The sentiment of the review is {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
The sentiment of the review is negative


In [27]:
new_review="The movie was ok but not that good."
sentiment=predict_ssentiment(new_review)
print(f"The sentiment of the review is {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
The sentiment of the review is negative


In [29]:
new_review="The movie was really fantastic, but boring at some points also"
sentiment=predict_ssentiment(new_review)
print(f"The sentiment of the review is {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
The sentiment of the review is negative
